In [2]:
import string
import numpy as np
import pandas as pd
import pyLDAvis.gensim
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases, TfidfModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from scipy.spatial import distance
from gensim.test.utils import datapath
from gensim.models import hdpmodel

C:\Users\ajain\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
articles = []

for l in range(1,4):
    k = 32
    if (l == 2):
        k=30     
    for i in range(1,k):
        dataP = pd.read_csv(str(l) + "_" + str(i) + "_P.csv")
        dataTH = pd.read_csv(str(l) + "_" + str(i) + "_TH.csv")
        dataTOI = pd.read_csv(str(l) + "_" + str(i) + "_TOI.csv")
        for j in range(len(dataP)):
            articles.append(dataP['Article'][j])
        for j in range(len(dataTH)):
            articles.append(dataTH['Content'][j])
        for j in range(len(dataTOI)):
            articles.append(dataTOI['Content'][j])
                     
     

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
corona =[]
dataTOI = pd.read_csv("corona.csv")
for j in range(len(dataTOI)):
    corona.append(dataTOI['content'][j])

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
import glob
path = r"C:\Users\ajain\Desktop\DATASET\TOI\*.csv"
for fname in glob.glob(path):
   df=pd.read_csv(fname)
   for j in range(len(df)):
        articles.append(df['Content'][j])

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
def standardize_data(articles):
    temp = []
    for article in articles:
        if type(article) is str:
            temp.append(article.lower().translate(str.maketrans('','' , string.punctuation)))
    for i in range(len(temp)):
        temp[i] = " ".join(temp[i].split())
    return temp

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
def word_preprocessing(articles):
    words = []
    for article in articles:
        words.append(word_tokenize(article))
    stop_words = set(stopwords.words("english"))

    for i in range(len(words)):
        words[i] = [word for word in words[i] if word not in stop_words]  

    for i in range(len(words)):
        words[i] = [token for token in words[i] if not token.isnumeric()] 

    lemmatizer = WordNetLemmatizer()
    for i in range(len(words)):
        words[i] = [lemmatizer.lemmatize(j) for j in words[i]]

    for i in range(len(words)): 
        words[i] = [token for token in words[i] if len(token) > 1]
    
    return words

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
articles = standardize_data(articles)
words = word_preprocessing(articles)

In [8]:
articles = standardize_data(corona)
words = word_preprocessing(articles)

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
def LDA_preprocessing(words):
    bigram = Phrases(words, min_count=5,threshold= 100)
    for idx in range(len(words)):
        for token in bigram[words[idx]]:
            if '_' in token:
                words[idx].append(token)    
    dictionary = Dictionary(words)
    dictionary.filter_extremes(no_below = 50 ,no_above = 0.30)
    corpus = [dictionary.doc2bow(doc) for doc in words]
    tfidf = TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    
    return dictionary, corpus, words

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
dictionary, corpus_tfidf, words = LDA_preprocessing(words)

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
len(dictionary)

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


7072

In [32]:
num_topics = 4
chunksize = 5000
passes = 20
iterations = 400
eval_every = None

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
model = LdaModel(corpus=corpus_tfidf, id2word=dictionary, chunksize=chunksize, 
                 alpha='auto', eta='auto', iterations=iterations, num_topics=num_topics,
                 passes=passes, eval_every=eval_every )

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
lda_display = pyLDAvis.gensim.prepare(model, corpus_tfidf, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
temp_file = datapath("modelf")
#model.save((temp_file))

modelf= LdaModel.load(temp_file)

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
lda_display = pyLDAvis.gensim.prepare(modelf, corpus_tfidf, dictionary, sort_topics=False)


C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
pyLDAvis.display(lda_display)


C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
#print('\nPerplexity: ', model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model, texts=words, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.6163591673326029


In [40]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [40]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus_tfidf, num_topics=6, id2word=dictionary)

In [46]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus_tfidf, texts=words, start=2, limit=20, step=6)


In [36]:
from scipy.spatial import distance

In [ ]:
distance.jensenshannon(doc1,doc2)

C:\Users\ajain\Anaconda3\lib\site-packages\gensim\test\test_data\modell


In [ ]:
from gensim.models import hdpmodel
hdp_model = hdpmodel.HdpModel(corpus=corpus_tfidf, id2word=dictionary)
hdp_model.print_topics(num_topics=-1)

In [ ]:
shown_topics = hdp_model.show_topics(num_topics=hdp_model.m_T, formatted=False)
topics_nos = [x[0] for x in shown_topics ]
weights = [ sum([item[1] for item in shown_topics[topicN][1]]) for topicN in topics_nos ]
ll= pd.DataFrame({'topic_id' : topics_nos, 'weight' : weights})

In [ ]:
for i in range(0,150):
    print(ll['weight'][i])
    print('\n')

In [45]:
train_vecs = []
for i in range(len(words)):
    top_topics = model.get_document_topics(corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(6)]
    train_vecs.append(topic_vec)

In [46]:
print(train_vecs[0])

[6.304063e-05, 5.930225e-05, 0.19166397, 4.6081892e-05, 0.80810606, 6.153711e-05]


In [24]:
test_articles = []

dataP = pd.read_csv("4_1_P.csv")
dataTH = pd.read_csv("4_1_TH.csv")
dataTOI = pd.read_csv("4_1_TOI.csv")
for j in range(len(dataP)):
    test_articles.append(dataP['Article'][j])
for j in range(len(dataTH)):
    test_articles.append(dataTH['Content'][j])
for j in range(len(dataTOI)):
    test_articles.append(dataTOI['Content'][j])
                     
 

In [25]:
test_articles = standardize_data(test_articles)
test = word_preprocessing(test_articles)

1158


In [32]:
testcorpus = [dictionary.doc2bow(doc) for doc in test]

In [10]:
corpus = [dictionary.doc2bow(doc) for doc in words]


C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
test_vecs = []
for i in range(len(words)):
    top_topics = modelf.get_document_topics(corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(6)]
    test_vecs.append(topic_vec)

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
topic0 = []
topic1 = []
topic2 = []
topic3 = []
topic4 = []
topic5 = []
matched_topic = [topic0, topic1, topic2, topic3, topic4, topic5]


C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
for i in range(len(test_vecs)):
    mx = -1
    for j in range(len(test_vecs[i])):
        mx = max(mx, test_vecs[i][j])
    
    for j in range(len(test_vecs[i])):
        if mx==test_vecs[i][j]:
            matched_topic[j].append(articles[i])
            break

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
len(matched_topic[0])

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


31970

In [18]:
dict = {'content' : matched_topic[0]}
df = pd.DataFrame(dict)  
    
# saving the dataframe  
df.to_csv('Corona.csv')

C:\Users\ajain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd


def convert_tag(tag):
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
   

    token = nltk.word_tokenize(doc)
    tag = nltk.pos_tag(token)
    nltk2wordnet = [(i[0], convert_tag(i[1])) for i in tag]
    output = [wn.synsets(i, z)[0] for i, z in nltk2wordnet if len(wn.synsets(i, z))>0]

    return output


def similarity_score(s1, s2):
   
    # Your Code Here
    list1 = []
    # For each synset in s1
    for a in s1:
        # finds the synset in s2 with the largest similarity value
        list1.append(max([i.path_similarity(a) for i in s2 if i.path_similarity(a) is not None]))

    output = sum(list1)/len(list1)

    return output


def document_path_similarity(doc1, doc2):
    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)
    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2






t = np.zeros((len(matched_topic[0]),len(matched_topic[0])))
t             

for i in range(len(matched_topic[0])):
              doc1 = test_articles[matched_topic[0][i]]
              for j in range(i, len(matched_topic[0])):
                  doc2 = test_articles[matched_topic[0][j]]
                  if i == j:
                      t[i][j]= -1
                      continue
                  t[i][j] = document_path_similarity(doc1, doc2)
                  t[j][i] = t[i][j]
                  print(t[i][j], end= '') 

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Create the Document Term Matrix
count_vectorizer = TfidfVectorizer(stop_words='english')
count_vectorizer = TfidfVectorizer()
sparse_matrix = count_vectorizer.fit_transform(matched_topic[0])

In [103]:
from sklearn.metrics.pairwise import cosine_similarity
a=[]
a= cosine_similarity( sparse_matrix, sparse_matrix)

In [ ]:
for i in range(len(a)):
    for k in range(len(a[i])):
        if a[i][k] > 0.5 and i!=k:
            print("(",i,k,")", end = ' ')
            
            
            

In [ ]:
print(matched_topic[3][5])

In [ ]:
print(matched_topic[3][602])